# Ensemble Learning

## Initial Imports

In [94]:
import warnings
warnings.filterwarnings('ignore')

In [95]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [96]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.preprocessing import StandardScaler, LabelEncoder

## Read the CSV and Perform Basic Data Cleaning

In [97]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)
df = pd.get_dummies(df, columns=["home_ownership"])
df = pd.get_dummies(df, columns=["verification_status"])
df = pd.get_dummies(df, columns=["pymnt_plan"])
df = pd.get_dummies(df, columns=["hardship_flag"])
df = pd.get_dummies(df, columns=["debt_settlement_flag"])
df = pd.get_dummies(df, columns=["initial_list_status"])
df = pd.get_dummies(df, columns=["application_type"])
df = pd.get_dummies(df, columns=["issue_d"])
df = pd.get_dummies(df, columns=["next_pymnt_d"])

# Preview the data
df.head()

,loan_amnt,int_rate,installment,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,debt_settlement_flag_N,initial_list_status_f,initial_list_status_w,application_type_Individual,application_type_Joint App,issue_d_Feb-2019,issue_d_Jan-2019,issue_d_Mar-2019,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019
0,10500.0,0.1719,375.35,66000.0,low_risk,27.24,0.0,0.0,8.0,0.0,...,1,0,1,1,0,0,0,1,0,1
1,25000.0,0.2000,929.09,105000.0,low_risk,20.23,0.0,0.0,17.0,1.0,...,1,0,1,1,0,0,0,1,0,1
2,20000.0,0.2000,529.88,56000.0,low_risk,24.26,0.0,0.0,8.0,0.0,...,1,0,1,1,0,0,0,1,0,1
3,10000.0,0.1640,353.55,92000.0,low_risk,31.44,0.0,1.0,10.0,1.0,...,1,0,1,1,0,0,0,1,0,1
4,22000.0,0.1474,520.39,52000.0,low_risk,18.76,0.0,1.0,14.0,0.0,...,1,0,1,1,0,0,0,1,0,1


## Split the Data into Training and Testing

In [98]:
# Create our features
X = df.copy().drop(['loan_status'], axis = 1)

# Create our target
y = df.copy()['loan_status']

In [99]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,debt_settlement_flag_N,initial_list_status_f,initial_list_status_w,application_type_Individual,application_type_Joint App,issue_d_Feb-2019,issue_d_Jan-2019,issue_d_Mar-2019,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,1.0,0.123879,0.876121,0.860340,0.139660,0.371696,0.451066,0.177238,0.383161,0.616839
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.0,0.329446,0.329446,0.346637,0.346637,0.483261,0.497603,0.381873,0.486161,0.486161
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,1.0,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,1.0,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,1.0,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.000000,1.000000
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [100]:
# Check the balance of our target values
# YOUR CODE HERE
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [101]:
# Split the X and y into X_train, X_test, y_train, y_test
# YOUR CODE HERE
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [102]:
# Create the StandardScaler instance
# YOUR CODE HERE
scaler = StandardScaler()

In [103]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
# YOUR CODE HERE
X_scaler = scaler.fit(X_train)

In [104]:
# Scale the training and testing data
# YOUR CODE HERE
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [105]:
# Resample the training data with the BalancedRandomForestClassifier
# YOUR CODE HERE
from imblearn import ensemble
model = ensemble.BalancedRandomForestClassifier()
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test)

In [106]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
from sklearn.metrics import balanced_accuracy_score
acc_score = balanced_accuracy_score(y_test, y_pred)
acc_score

0.5871934305217239

In [107]:
# Display the confusion matrix
# YOUR CODE HERE

confusion_matrix(y_test, y_pred)

array([[   54,    47],
       [ 6162, 10942]])

In [108]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.53      0.64      0.02      0.58      0.34       101
   low_risk       1.00      0.64      0.53      0.78      0.58      0.35     17104

avg / total       0.99      0.64      0.54      0.77      0.58      0.35     17205



In [109]:
# List the features sorted in descending order by feature importance
# YOUR CODE HERE
importances = model.feature_importances_
importances_df = pd.DataFrame(sorted(zip(model.feature_importances_, X_train_scaled.columns), reverse=True))
importances_df.set_index(importances_df[1], inplace=True)
importances_df.drop(columns=1, inplace=True)
importances_df.rename(columns={0: 'Feature Importances'}, inplace=True)
importances_sorted = importances_df.sort_values(by='Feature Importances', ascending = False)
importances_sorted.head()


AttributeError: 'numpy.ndarray' object has no attribute 'columns'

### Easy Ensemble Classifier

In [ ]:
# Train the Classifier
# YOUR CODE HERE
model = ensemble.EasyEnsembleClassifier()
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test_scaled)

In [ ]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
acc_score = balanced_accuracy_score(y_test, y_pred)
acc_score

In [ ]:
# Display the confusion matrix
# YOUR CODE HERE
confusion_matrix(y_test, y_pred)

In [110]:
# Print the imbalanced classification report
# YOUR CODE HERE
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.01      0.53      0.64      0.02      0.58      0.34       101
   low_risk       1.00      0.64      0.53      0.78      0.58      0.35     17104

avg / total       0.99      0.64      0.54      0.77      0.58      0.35     17205



### Final Questions

1. Which model had the best balanced accuracy score?

    The ensemble classifier had a significantly better accuracy score

2. Which model had the best recall score?

    The ensemble classifier had a significantly better recall score

3. Which model had the best geometric mean score?

    The ensemble classifier had a significantly better eometric mean score

4. What are the top three features?

total_pymnt_inv
total_rec_prncp
total_pymnt